In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain

ProjDir = rel_path("..", "scripts", "12")

d = CSV.read(rel_path( "..", "data",  "Kline.csv"), delim=';');
size(d) # Should be 10x5

(10, 5)

New col logpop, set log() for population data

In [2]:
d[:logpop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

first(d, 5)

m12_6_1 = "
data{
    int total_tools[10];
    real logpop[10];
    int society[10];
}
parameters{
    real a;
    real bp;
    vector[10] a_society;
    real<lower=0> sigma_society;
}
model{
    vector[10] mu;
    sigma_society ~ cauchy( 0 , 1 );
    a_society ~ normal( 0 , sigma_society );
    bp ~ normal( 0 , 1 );
    a ~ normal( 0 , 10 );
    for ( i in 1:10 ) {
        mu[i] = a + a_society[society[i]] + bp * logpop[i];
        mu[i] = exp(mu[i]);
    }
    total_tools ~ poisson( mu );
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [3]:
stanmodel = Stanmodel(name="m12.6.1",  model=m12_6_1, output_format=:mcmcchain);


File /Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan will be updated.



Input data for cmdstan

In [4]:
m12_6_1_data = Dict("total_tools" => d[:total_tools], "logpop" => d[:logpop], "society" => d[:society]);

Sample using cmdstan

In [5]:
rc, chn, cnames = stan(stanmodel, m12_6_1_data, ProjDir, diagnostics=false, summary=false, CmdStanDir=CMDSTAN_HOME);

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.

Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in '/Users/rob/.julia/dev/StatisticalRethinking/notebooks/12/tmp/m12.6.1.stan' at line 15)

If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
but if this warning occurs often then your model may be either severely ill-conditioned 

Describe the draws

In [6]:
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
                   Mean           SD         Naive SE        MCSE         ESS    
         lp__  927.857644500  3.025980554 0.04784495354 0.10362610888  852.695329
accept_stat__    0.934880775  0.087598400 0.00138505232 0.00170492182 1000.000000
   stepsize__    0.050741500  0.002884818 0.00004561298 0.00046188285   39.009752
  treedepth__    5.905500000  0.427920003 0.00676600933 0.01101834135 1000.000000
 n_leapfrog__   72.332000000 33.269422743 0.52603576153 0.79727510292 1000.000000
  divergent__    0.000000000  0.000000000 0.00000000000 0.00000000000         NaN
     energy__ -921.368807250  3.950867744 0.06246870403 0.13466897857  860.696132
            a    1.109580272  0.745582754 0.01178869843 0.02159483156 1000.000000
           bp    0.259711938  0.080091373 0.00126635580 0.00231493917 1000.000000
  a_society.1   -0.203999044  0.254121303 0.00401801060 0.006671

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*